In [184]:
import sklearn as sk
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

We will


1.   load the iris dataseta and write it to a sql database
2.   load the data from the sql database and run a simple k-means clustering on it.
3. write the results to the sql database


In [185]:
irisdata = load_iris()

In [187]:
def query(db,query):
  connection = sql.connect(db)
  return connection.cursor().execute(query).fetchall()
  connection.commit()
  connection.close()

In [230]:
query("IRIS.db", "CREATE TABLE Iris_Table (Id INTEGER PRIMARY KEY, sepal_length INTEGER, sepal_width INTEGER, \
petal_length INTEGER, petal_width INTEGER,Target INTEGER)");
#.Execute() can't pass variables to table names or column names (anti injection)
#ALTER TABLE ADD can only add 1 column at a time, so create them at CREATE TABLE

In [229]:
target = irisdata.target.astype(np.float64) #Convert the dtype to the same as values or sql gets angry
idx = np.arange(150).astype(np.float64) #We add a index column for id purposes #This will be PRIMARY KEY
values = [tuple([idx[i]])+tuple(irisdata.data[i])+ tuple([target[i]]) for i in range(len(irisdata.target))]
#List of 5-tuples
print(values[0])

(0.0, 5.1, 3.5, 1.4, 0.2, 0.0)


In [231]:
connection = sql.connect("IRIS.db")
for v in values:
  connection.cursor().execute("INSERT INTO Iris_Table (Id, sepal_length,\
   sepal_width, petal_length, petal_width, Target) VALUES (?,?,?,?,?,?)",v);
connection.commit();
connection.close();

In [254]:
data = query("IRIS.db", "SELECT sepal_length,sepal_width,petal_length, petal_width FROM Iris_Table")
target = query("IRIS.db", "SELECT target from Iris_Table")
data = [list(d) for d in data]
target = [t[0] for t in target]

In [262]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters = 3);
#Since there are 3 types of irises in the dataset, we expect 3 clusters
#of course since this algorithm is unsupervised, it has no idea which cluster label
#belongs to which iris.
model.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [264]:
#Since the data was already ordered, we see that
#the labelling scheme of KMeans matched that of the data
model.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1], dtype=int32)

In [271]:
acc = np.sum(model.labels_ == target)/len(target)
print("Accuracy : {}".format(acc))

Accuracy : 0.8933333333333333


In [285]:
labels = model.labels_.astype(np.float64)
labels = [(labels[i],idx[i]) for i in range(len(target))]

In [ ]:
query("IRIS.db", "ALTER TABLE Iris_Table ADD Kmeans_prediction INTEGER");

In [288]:
connection = sql.connect("IRIS.db");
connection.cursor().executemany("UPDATE Iris_Table SET Kmeans_prediction = ? WHERE ID = ?", labels)
connection.commit();
connection.close();